In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
E = 210e9
nu = 0.3
YS = 240e6
b = 0.0001
beta = np.pi/12
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [5]:
def calculateK (press,x,shift,a1,a2,theta1,theta2,t,c):
    mapdl.clear()
    mapdl.prep7()
    m0 = mapdl.k("", 0,-0.1)
    m1 = mapdl.k("",0,0.1)
    m6 = mapdl.k("", 0.1, 0.1)
    m7 = mapdl.k("", 0.1, -0.1)
    main = mapdl.a(m0,m1,m6,m7)
    us1 = np.tan(theta1)
    us2 = np.tan(theta1+beta)# lower side of crack 1
    us3 = np.tan(theta1-beta)# upper side of crack 2

    d = a1*(np.sin(theta1)-np.cos(theta1)*us2)
    e = a1*(np.sin(theta1)-np.cos(theta1)*us3)

    x1 = (b+d)/(us1-us2)
    y1 = us1*x1-b 
    x3 = (b-e)/(us3-us1)
    y3 = us1*x3+b 
    
    # Crack geometry for Crack 1
    k0 = mapdl.k("",0,-b +shift)
    k1 = mapdl.k("",x1,y1+shift)
    k2 = mapdl.k("",a1*np.cos(theta1),a1*np.sin(theta1)+shift)
    k3 = mapdl.k("",x3,y3+shift)
    k4 = mapdl.k("",0,b+shift)
    crack1 = mapdl.a(k0,k1,k2,k3,k4)
    aout = mapdl.asba(main, crack1)

    ### Geometry for Crack 2

    ls1 = np.tan(theta2)
    ls2 = np.tan(theta2+beta)# lower side of crack 1
    ls3 = np.tan(theta2-beta)# upper side of crack 2

    d = a2*(np.sin(theta2)-np.cos(theta2)*ls2)
    e = a2*(np.sin(theta2)-np.cos(theta2)*ls3)

    x1 = (b+d)/(ls1-ls2)
    y1 = ls1*x1-b 
    x3 = (b-e)/(ls3-ls1)
    y3 = ls1*x3+b 


    k0 = mapdl.k("",0,-b -shift)
    k1 = mapdl.k("",x1,y1-shift)
    k2 = mapdl.k("",a2*np.cos(theta2),a2*np.sin(theta2)-shift)
    k3 = mapdl.k("",x3,y3-shift)
    k4 = mapdl.k("",0,b-shift)
    crack2 = mapdl.a(k0,k1,k2,k3,k4)
    aout = mapdl.asba(aout, crack2)
    mapdl.vext(aout, dz=t)
   
    mapdl.et(1,"SOLID285")
    # Define the Mesh
    mapdl.smrtsize(2)
    mapdl.aesize("ALL", 0.003)
    mapdl.vmesh(1)
    #mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")
    # Using the SI units

    mapdl.units("SI")  # SI - International system (m, kg, s, K).
    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio
    # Fix the lower side.
    mapdl.nsel("S", "LOC", "Y", -0.1)
    mapdl.d("ALL", "UY")
    mapdl.d("ALL", "UZ")
    mapdl.d("ALL", "UX")

    mapdl.nsel("S", "LOC", "Y", 0.1)
    mapdl.cp(5, "UX", "ALL")

    mapdl.asel("S",'LOC',"Y",0.1)
    mapdl.sfa('ALL', '', 'PRES', -press)
    mapdl.allsel(mute=True)
    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()
    mapdl.finish(mute=True)
    
    # List parameters to post-process
    if c==1:
        mapdl.post1()
        mapdl.path('P1',3,5)
        node1 = mapdl.queries.node(a1*np.cos(theta1),a1*np.sin(theta1)+shift,0)
        xs1 = 0.6*a1
        ys1 = us1*xs1-b+shift
        node2 = mapdl.queries.node(xs1,ys1,0)
        node3 = mapdl.queries.node(0,-b+shift,0)
        mapdl.ppath(1, node1)
        mapdl.ppath(2, node2)
        mapdl.ppath(3, node3)
        mapdl.local(11,0,a1*np.cos(theta1),a1*np.sin(theta1)+shift,0,theta1*180/np.pi)
        mapdl.csys(11)
        k1 = mapdl.kcalc(0,1,0,0)
        result = mapdl.result
        words = k1.split(' ')
        # Extracting the K1 value from the String output
        for i in range(len(words)):
            if words[i]=='KI':
                x=i
        K1 = words[x+3][:-1] 
        print(K1)
    elif c==2:
        mapdl.post1()
        mapdl.path('P2',3,5)
        node4 = mapdl.queries.node(a2*np.cos(theta2),a2*np.sin(theta2)-shift,0)
        xs2 = 0.6*a2
        ys2 = ls1*xs2+b-shift
        node5 = mapdl.queries.node(xs2,ys2,0)
        node6 = mapdl.queries.node(0,b-shift,0)
        mapdl.ppath(1, node4)
        mapdl.ppath(2, node5)
        mapdl.ppath(3, node6)
        mapdl.local(12,0,a2*np.cos(theta2),a2*np.sin(theta2)-shift,0,theta2*180/np.pi)
        mapdl.post1()
        mapdl.csys(12)
        k2 = mapdl.kcalc(0,1,0,0)

        result = mapdl.result
        words = k2.split(' ')
        # Extracting the K1 value from the String output
        for i in range(len(words)):
            if words[i]=='KI':
                x=i
        K1 = words[x+3][:-1] 
        print(K1)
   
    return K1


In [6]:
A = calculateK(250e6,0.005,0.01,0.02,0.02,np.pi/10,-np.pi/9,0.005,2)

0.67095E+08
